## 5.3

对于习题 4.5 中的 1991 年我国各省、区、市城镇居民 8 个消费指标数据（见表 4.9），设前 20 个省份为第 1 类 $C_1$，第 21–27 号省份（即福建、……、北京）为第 2 类 $C_2$，最后二个省份（西藏、上海、广东）待判。

### (1)
进行距离判别，给出决策线及一次判别函数，并计算误判率的似然估计与交叉确认估计

### (2)
设两总体服从正态分布，先验概率按比例分配且误判损失相同。分别在两总体的协方差矩阵相等和不等的情况下，进行Bayes 判别分析，并计算判别率的似然估计与交叉确认估计

### (3)
在上述各情况下，试判别西藏、上海、广东各属哪一类

In [ ]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

data_5_3 = {
    "省(区、市)": [
        "山西", "内蒙古", "吉林", "黑龙江", "河南", "甘肃", "青海", "河北",
        "陕西", "宁夏", "新疆", "湖北", "云南", "湖南",
        "安徽", "贵州", "辽宁", "四川", "山东", "江西", "福建", "广西",
        "海南", "天津", "江苏", "浙江", "北京", "西藏", "上海", "广东"
    ],
    "x1": [
        8.35, 9.25, 8.19, 7.73, 9.42, 9.16, 10.06, 9.09,
        9.41, 8.70, 6.93, 8.67, 9.98, 6.77,
        8.14, 7.67, 7.90, 7.18, 8.82, 6.25, 10.60, 7.27,
        13.45, 10.85, 7.21, 7.68, 7.78, 7.94, 8.28, 12.47
    ],
    "x2": [
        23.53, 23.75, 30.50, 29.20, 27.93, 27.98, 28.64, 28.12,
        28.20, 28.12, 29.85, 36.05, 37.69, 38.69,
        37.75, 35.71, 39.77, 40.91, 33.70, 35.02, 52.41, 52.65,
        55.85, 44.68, 45.79, 50.37, 48.44, 39.65, 64.34, 76.39
    ],
    "x3": [
        7.51, 6.61, 4.72, 5.42, 8.20, 9.01, 10.52, 7.40,
        5.77, 7.21, 4.54, 7.31, 7.01, 6.01,
        9.61, 8.04, 8.49, 7.32, 7.59, 4.72, 7.70, 3.84,
        5.50, 7.32, 7.66, 11.35, 8.00, 20.97, 8.00, 5.52
    ],
    "x4": [
        8.62, 9.19, 9.78, 9.43, 8.14, 9.32, 10.05, 9.62,
        10.80, 10.53, 9.49, 7.75, 8.94, 8.82,
        8.49, 8.31, 12.94, 8.94, 10.98, 6.28, 9.98, 9.16,
        7.45, 14.51, 10.36, 13.30, 20.51, 20.82, 22.22, 11.24
    ],
    "x5": [
        17.42, 17.77, 16.28, 19.29, 16.17, 15.99, 16.18, 17.26,
        16.36, 19.45, 16.62, 16.67, 16.15, 14.79,
        13.15, 15.13, 19.27, 17.60, 18.82, 10.03, 12.53, 13.03,
        9.55, 17.13, 16.56, 19.25, 22.12, 22.52, 20.06, 14.52
    ],
    "x6": [
        10.00, 10.48, 7.60, 8.49, 9.42, 9.10, 8.39, 11.12,
        11.56, 13.30, 10.65, 11.68, 11.08, 11.44,
        9.76, 7.76, 11.05, 12.75, 14.73, 7.15, 11.70, 15.26,
        9.52, 12.08, 12.86, 14.59, 15.73, 12.41, 15.12, 22.00
    ],
    "x7": [
        1.04, 1.72, 2.52, 2.52, 1.55, 1.82, 1.96, 2.49,
        1.53, 1.66, 1.88, 2.38, 0.83, 1.74,
        1.28, 1.41, 2.04, 1.14, 1.78, 1.93, 2.31, 1.98,
        2.21, 1.26, 2.25, 2.75, 1.15, 1.75, 0.72, 5.46
    ],
    "x8": [
        11.21, 10.51, 10.32, 10.00, 9.76, 11.35, 10.81, 12.65,
        12.17, 11.96, 13.61, 12.88, 11.67, 13.23,
        11.28, 13.25, 13.29, 14.80, 10.10, 10.39, 14.69, 14.57,
        16.30, 11.57, 11.69, 14.87, 16.61,  7.90, 22.89, 25.50
    ]
}
df = pd.DataFrame(data_5_3).set_index("省(区、市)")

provinces = df.index.tolist()

# 前 20 个省为 C1，第 21–27 为 C2，最后 3 个为待判
class1 = provinces[:20]    # C1
class2 = provinces[20:27]  # C2
unknown = provinces[27:]   # “西藏”、“上海”、“广东”

# 构造训练集与标签
df_train = df.loc[class1 + class2]
X_train = df_train.values              # 训练样本矩阵 (27×8)
y_train = np.array([1]*20 + [2]*7)      # 对应类别向量 (27,)

# 待判样本
X_unknown = df.loc[unknown].values

# 计算两类均值向量
mu1 = df.loc[class1].mean().values   # C1 的 8 维均值
mu2 = df.loc[class2].mean().values   # C2 的 8 维均值

# (1) 距离判别（Euclidean Distance）
def euclid_classify(x, mu1, mu2):
    d1 = np.linalg.norm(x - mu1)
    d2 = np.linalg.norm(x - mu2)
    return 1 if d1 < d2 else 2

# 训练样本的“似然估计”误判率
pred_train_euclid = [euclid_classify(x, mu1, mu2) for x in X_train]
error_train_euclid = np.mean(pred_train_euclid != y_train)

# 留一交叉验证误判率
errors_cv_euclid = 0
for i in range(len(X_train)):
    # 留出第 i 个样本
    Xi = X_train[i]
    yi = y_train[i]
    names = class1 + class2
    mask = np.arange(len(X_train)) != i
    # 重新计算两类均值
    class1_loo = [names[j] for j in range(len(names)) if mask[j] and y_train[j] == 1]
    class2_loo = [names[j] for j in range(len(names)) if mask[j] and y_train[j] == 2]
    mu1_loo = df.loc[class1_loo].mean().values
    mu2_loo = df.loc[class2_loo].mean().values
    pred_i = euclid_classify(Xi, mu1_loo, mu2_loo)
    if pred_i != yi:
        errors_cv_euclid += 1
error_cv_euclid = errors_cv_euclid / len(X_train)

# 对“西藏、上海、广东”进行判别
pred_unknown_euclid = [euclid_classify(x, mu1, mu2) for x in X_unknown]


# (2) Bayes 判别 2.1: LDA（协方差相等，先验按比例）

lda = LinearDiscriminantAnalysis(store_covariance=True)
lda.fit(X_train, y_train)

# 训练误判率
pred_train_lda = lda.predict(X_train)
error_train_lda = np.mean(pred_train_lda != y_train)

# 留一交叉验证误判率
errors_cv_lda = 0
for i in range(len(X_train)):
    Xi = X_train[i].reshape(1, -1)
    yi = y_train[i]
    mask = np.arange(len(X_train)) != i
    lda_loo = LinearDiscriminantAnalysis(store_covariance=True)
    lda_loo.fit(X_train[mask], y_train[mask])
    if lda_loo.predict(Xi)[0] != yi:
        errors_cv_lda += 1
error_cv_lda = errors_cv_lda / len(X_train)

# 对“西藏、上海、广东”进行判别
pred_unknown_lda = lda.predict(X_unknown)


# (3) Bayes 判别 2.2: QDA（协方差不等，先验按比例）

qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

# 训练误判率
pred_train_qda = qda.predict(X_train)
error_train_qda = np.mean(pred_train_qda != y_train)

# 留一交叉验证误判率
errors_cv_qda = 0
for i in range(len(X_train)):
    Xi = X_train[i].reshape(1, -1)
    yi = y_train[i]
    mask = np.arange(len(X_train)) != i
    qda_loo = QuadraticDiscriminantAnalysis()
    qda_loo.fit(X_train[mask], y_train[mask])
    if qda_loo.predict(Xi)[0] != yi:
        errors_cv_qda += 1
error_cv_qda = errors_cv_qda / len(X_train)

# 对“西藏、上海、广东”进行判别
pred_unknown_qda = qda.predict(X_unknown)

print("距离判别 (Euclidean)")
print(f"训练误判率 = {error_train_euclid:.4f}")
print(f"交叉验证误判率 = {error_cv_euclid:.4f}")
print("未知省份分类:", dict(zip(unknown, pred_unknown_euclid)))

print("\nLDA (协方差矩等)")
print(f"训练误判率 = {error_train_lda:.4f}")
print(f"交叉验证误判率 = {error_cv_lda:.4f}")
print("未知省份分类:", dict(zip(unknown, pred_unknown_lda)))

print("\nQDA (协方差不等) ")
print(f"训练误判率 = {error_train_qda:.4f}")
print(f"交叉验证误判率 = {error_cv_qda:.4f}")
print("未知省份分类:", dict(zip(unknown, pred_unknown_qda)))


距离判别 (Euclidean)
训练误判率 = 0.0000
交叉验证误判率 = 0.0370
未知省份分类: {'西藏': 1, '上海': 2, '广东': 2}

LDA (协方差矩等)
训练误判率 = 0.0000
交叉验证误判率 = 0.0741
未知省份分类: {'西藏': np.int64(2), '上海': np.int64(2), '广东': np.int64(2)}

QDA (协方差不等) 
训练误判率 = 0.0000
交叉验证误判率 = 0.2593
未知省份分类: {'西藏': np.int64(1), '上海': np.int64(1), '广东': np.int64(1)}


c:\Users\Mako\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
c:\Users\Mako\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
c:\Users\Mako\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
c:\Users\Mako\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not 

**距离判别（Euclidean）**

* **判别规则**：记两类均值向量
  $\displaystyle \mu_1=\frac1{20}\sum_{i\in C_1}x_i,\quad \mu_2=\frac1{7}\sum_{i\in C_2}x_i$。
  对样本 $x$，若 $\|x-\mu_1\|<\|x-\mu_2\|$ ⇒ 判 $C_1$，否则判 $C_2$。
* **误判率**

  * 训练集（“似然估计”）：0.0000
  * 留一交叉验证：0.0370
* **未知省份分类**

  * 西藏 → C1
  * 上海 → C2
  * 广东 → C2

---

**LDA（等协方差 Bayes 判别）**

* **判别函数**：

  $$
    \delta_k(x) = -\tfrac12(x-\mu_k)^\top\Sigma^{-1}(x-\mu_k)+\ln\pi_k,\quad k=1,2,
  $$

  其中 $\mu_k$ 为各类均值，$\Sigma$ 为合并协方差，$\pi_k$ 取样本比例。判令 $\delta_1(x)>\delta_2(x)$ ⇒ C1，否则 C2。
* **误判率**

  * 训练集：0.0000
  * 留一交叉验证：0.0741
* **未知省份分类**

  * 西藏 → C2
  * 上海 → C2
  * 广东 → C2

---

**QDA（不等协方差 Bayes 判别）**

* **判别函数**：

  $$
    \delta_k(x)=-\tfrac12\ln|\Sigma_k|-\tfrac12(x-\mu_k)^\top\Sigma_k^{-1}(x-\mu_k)+\ln\pi_k,\quad k=1,2.
  $$

  令 $\delta_1(x)>\delta_2(x)$ ⇒ C1，否则 C2，$\Sigma_k$ 为各类协方差估计，$\pi_k$ 同上。
* **误判率**

  * 训练集：0.0000
  * 留一交叉验证：0.2593
* **未知省份分类**

  * 西藏 → C1
  * 上海 → C1
  * 广东 → C1


## 5.5
考察鸢尾属植物中三个不同品种的花的如下四个形状指标：  
$ X_1：萼片长度； X_2：萼片宽度； X_3：花瓣长度； X_4：花瓣宽度。  $

从这三个品种（记为 1、2、3）各选取 50 株，测得上述指标的取值如表 5.10 所示。假定三个品种的这 4 个指标均服从 4 维正态分布，且先验概率相等，按下列要求进行 Bayes 判别分析：


In [24]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.model_selection import LeaveOneOut
data_5_5 = [
    [1,   1, 50, 33, 14,  2],
    [2,   1, 46, 34, 14,  3],
    [3,   1, 46, 36, 10,  2],
    [4,   1, 51, 33, 17,  5],
    [5,   1, 55, 35, 13,  2],
    [6,   1, 48, 31, 16,  2],
    [7,   1, 52, 34, 14,  2],
    [8,   1, 49, 36, 14,  1],
    [9,   1, 44, 32, 13,  2],
    [10,  1, 50, 35, 16,  3],
    [11,  1, 44, 30, 13,  2],
    [12,  1, 47, 32, 16,  2],
    [13,  1, 48, 30, 14,  3],
    [14,  1, 51, 38, 16,  2],
    [15,  1, 48, 34, 19,  2],
    [16,  1, 50, 30, 16,  2],
    [17,  1, 50, 32, 12,  2],
    [18,  1, 43, 30, 11,  1],
    [19,  1, 58, 40, 12,  2],
    [20,  1, 51, 38, 19,  4],
    [21,  1, 49, 30, 14,  2],
    [22,  1, 51, 35, 14,  2],
    [23,  1, 50, 34, 16,  4],
    [24,  1, 46, 32, 14,  2],
    [25,  1, 57, 44, 15,  4],
    [26,  1, 50, 36, 14,  2],
    [27,  1, 54, 34, 15,  4],
    [28,  1, 52, 42, 15,  1],
    [29,  1, 55, 42, 14,  2],
    [30,  1, 49, 31, 15,  2],
    [31,  1, 54, 39, 17,  4],
    [32,  1, 50, 34, 15,  2],
    [33,  1, 44, 29, 14,  2],
    [34,  1, 47, 32, 13,  2],
    [35,  1, 46, 31, 15,  2],
    [36,  1, 51, 34, 15,  2],
    [37,  1, 50, 35, 13,  3],
    [38,  1, 49, 31, 15,  1],
    [39,  1, 54, 37, 15,  2],
    [40,  1, 54, 39, 13,  4],
    [41,  1, 51, 35, 14,  3],
    [42,  1, 48, 34, 16,  2],
    [43,  1, 48, 30, 14,  1],
    [44,  1, 45, 23, 13,  3],
    [45,  1, 57, 38, 17,  3],
    [46,  1, 51, 38, 15,  3],
    [47,  1, 54, 34, 17,  2],
    [48,  1, 51, 37, 15,  4],
    [49,  1, 52, 35, 15,  2],
    [50,  1, 53, 37, 15,  2],
    [51,  2, 65, 28, 46, 15],
    [52,  2, 62, 22, 45, 15],
    [53,  2, 59, 32, 48, 18],
    [54,  2, 61, 30, 46, 14],
    [55,  2, 60, 27, 51, 16],
    [56,  2, 56, 25, 39, 11],
    [57,  2, 57, 28, 45, 13],
    [58,  2, 63, 33, 47, 16],
    [59,  2, 70, 32, 47, 14],
    [60,  2, 64, 32, 45, 15],
    [61,  2, 61, 28, 40, 13],
    [62,  2, 55, 24, 38, 11],
    [63,  2, 54, 30, 45, 15],
    [64,  2, 58, 26, 40, 12],
    [65,  2, 55, 26, 44, 12],
    [66,  2, 50, 23, 33, 10],
    [67,  2, 67, 31, 44, 14],
    [68,  2, 56, 30, 45, 15],
    [69,  2, 58, 27, 41, 10],
    [70,  2, 60, 29, 45, 15],
    [71,  2, 57, 26, 35, 10],
    [72,  2, 57, 19, 42, 13],
    [73,  2, 49, 24, 33, 10],
    [74,  2, 56, 27, 42, 13],
    [75,  2, 57, 30, 42, 12],
    [76,  2, 66, 29, 46, 13],
    [77,  2, 52, 27, 39, 14],
    [78,  2, 60, 34, 45, 16],
    [79,  2, 50, 20, 35, 10],
    [80,  2, 55, 24, 37, 10],
    [81,  2, 58, 27, 39, 12],
    [82,  2, 62, 29, 43, 13],
    [83,  2, 59, 30, 42, 15],
    [84,  2, 60, 22, 40, 10],
    [85,  2, 67, 31, 47, 15],
    [86,  2, 63, 23, 44, 13],
    [87,  2, 56, 30, 41, 13],
    [88,  2, 63, 25, 49, 15],
    [89,  2, 61, 28, 47, 12],
    [90,  2, 64, 29, 43, 13],
    [91,  2, 51, 25, 30, 11],
    [92,  2, 57, 28, 41, 13],
    [93,  2, 61, 29, 47, 14],
    [94,  2, 56, 29, 36, 13],
    [95,  2, 69, 31, 49, 15],
    [96,  2, 55, 25, 40, 13],
    [97,  2, 55, 23, 40, 13],
    [98,  2, 66, 30, 44, 14],
    [99,  2, 68, 28, 48, 14],
    [100, 2, 67, 30, 50, 17],
    [101, 3, 64, 28, 56, 22],
    [102, 3, 67, 31, 56, 24],
    [103, 3, 63, 28, 51, 15],
    [104, 3, 69, 31, 51, 23],
    [105, 3, 65, 30, 52, 20],
    [106, 3, 65, 30, 55, 18],
    [107, 3, 58, 27, 51, 19],
    [108, 3, 68, 32, 59, 23],
    [109, 3, 62, 34, 54, 23],
    [110, 3, 77, 38, 67, 22],
    [111, 3, 67, 33, 57, 25],
    [112, 3, 76, 30, 66, 21],
    [113, 3, 49, 25, 45, 17],
    [114, 3, 67, 30, 52, 23],
    [115, 3, 59, 30, 51, 18],
    [116, 3, 63, 25, 50, 19],
    [117, 3, 64, 32, 53, 23],
    [118, 3, 79, 38, 64, 20],
    [119, 3, 67, 33, 57, 21],
    [120, 3, 77, 28, 67, 20],
    [121, 3, 63, 27, 49, 18],
    [122, 3, 72, 32, 60, 18],
    [123, 3, 61, 30, 49, 18],
    [124, 3, 61, 26, 56, 14],
    [125, 3, 64, 28, 56, 21],
    [126, 3, 62, 28, 48, 18],
    [127, 3, 77, 30, 61, 23],
    [128, 3, 63, 34, 56, 24],
    [129, 3, 58, 27, 51, 19],
    [130, 3, 72, 30, 58, 16],
    [131, 3, 71, 30, 59, 21],
    [132, 3, 64, 31, 55, 18],
    [133, 3, 60, 30, 48, 18],
    [134, 3, 63, 29, 56, 18],
    [135, 3, 77, 26, 69, 23],
    [136, 3, 60, 22, 50, 15],
    [137, 3, 69, 32, 57, 23],
    [138, 3, 74, 28, 61, 19],
    [139, 3, 56, 28, 49, 20],
    [140, 3, 73, 29, 63, 18],
    [141, 3, 67, 25, 58, 18],
    [142, 3, 65, 30, 58, 22],
    [143, 3, 69, 31, 54, 21],
    [144, 3, 72, 36, 61, 25],
    [145, 3, 65, 32, 51, 20],
    [146, 3, 64, 27, 53, 19],
    [147, 3, 68, 30, 55, 21],
    [148, 3, 57, 25, 50, 20],
    [149, 3, 58, 28, 51, 24],
    [150, 3, 63, 33, 60, 25],
]

columns = ['编号', '品种', 'x1', 'x2', 'x3', 'x4']

df = pd.DataFrame(data_5_5, columns=columns)
df.set_index('编号', inplace=True)




### (1)
只考虑指标 $X_{2}$ 和 $X_{4}$，并假定各总体协方差矩阵不全相等，给出误判率的回代估计和交叉确认估计；  


In [25]:
X_24 = df[['x2', 'x4']].values  # 仅用 X2, X4
X_4all = df[['x1', 'x2', 'x3', 'x4']].values  # 全部 4 维
y = df['品种'].values

# 留一法 (LOO) 误判率计算函数
def loocv_error(X, y, model_type='QDA'):
    loo = LeaveOneOut()
    errors = 0
    for train_idx, test_idx in loo.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        if model_type == 'QDA':
            clf = QuadraticDiscriminantAnalysis(store_covariance=True)
        else:
            clf = LinearDiscriminantAnalysis()
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        if pred[0] != y_test[0]:
            errors += 1
    return errors / len(y)

# 训练 QDA 模型并计算回代误判率
qda_24 = QuadraticDiscriminantAnalysis(store_covariance=True)
qda_24.fit(X_24, y)
resub_24_qda = np.mean(qda_24.predict(X_24) != y)

# 留一法 CV 误判率
cv_24_qda = loocv_error(X_24, y, model_type='QDA')

print(f"QDA 回代误判率 (X2,X4): {resub_24_qda:.4f}")
print(f"QDA LOO-CV 误判率 (X2,X4): {cv_24_qda:.4f}")
print()


QDA 回代误判率 (X2,X4): 0.0467
QDA LOO-CV 误判率 (X2,X4): 0.0467



### (2)
只考虑指标 $X_{2}$ 和 $X_{4}$，并假定各总体协方差矩阵相等，写出线性判别函数，给出误判率的回代估计和交叉确认估计，并与（1）中结果比较；  


In [26]:
lda_24 = LinearDiscriminantAnalysis()
lda_24.fit(X_24, y)
resub_24_lda = np.mean(lda_24.predict(X_24) != y)
cv_24_lda = loocv_error(X_24, y, model_type='LDA')

print(f"LDA 回代误判率 (X2,X4): {resub_24_lda:.4f}")
print(f"LDA LOO-CV 误判率 (X2,X4): {cv_24_lda:.4f}")
print()


LDA 回代误判率 (X2,X4): 0.0333
LDA LOO-CV 误判率 (X2,X4): 0.0400



### (3)
假定有新样品 $\mathbf{x}_{0}=(x_{2},\,x_{4})'=(35,\,18)'$，在（1）、（2）之下，该样品分别判归哪个总体？  


In [27]:
x0 = np.array([[35, 18]])
pred_qda_x0 = qda_24.predict(x0)[0]
pred_lda_x0 = lda_24.predict(x0)[0]

lda_4 = LinearDiscriminantAnalysis()
lda_4.fit(X_4all, y)
resub_4_lda = np.mean(lda_4.predict(X_4all) != y)
cv_4_lda = loocv_error(X_4all, y, model_type='LDA')

print(f"QDA 判归 (35,18): 品种 {pred_qda_x0}")
print(f"LDA 判归 (35,18): 品种 {pred_lda_x0}")
print()


QDA 判归 (35,18): 品种 2
LDA 判归 (35,18): 品种 3



### (4)
利用全部 4 个指标重复（1）和（2）的分析，结果如何？是否所用指标越多，分类效果越好？再尝试其他几种指标组合，情况又如何？


In [28]:
qda_4 = QuadraticDiscriminantAnalysis(store_covariance=True)
qda_4.fit(X_4all, y)
resub_4_qda = np.mean(qda_4.predict(X_4all) != y)
cv_4_qda = loocv_error(X_4all, y, model_type='QDA')

print(f"QDA 回代误判率 (4 维): {resub_4_qda:.4f}")
print(f"QDA LOO-CV 误判率 (4 维): {cv_4_qda:.4f}")
print(f"LDA 回代误判率 (4 维): {resub_4_lda:.4f}")
print(f"LDA LOO-CV 误判率 (4 维): {cv_4_lda:.4f}")

QDA 回代误判率 (4 维): 0.0200
QDA LOO-CV 误判率 (4 维): 0.0200
LDA 回代误判率 (4 维): 0.0200
LDA LOO-CV 误判率 (4 维): 0.0200
